In [50]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.9.2


# Artificial Neural Networks(ANN)

# Importing Libraries

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

[Dataset](https://www.kaggle.com/datasets/shubh0799/churn-modelling) from Kaggle

In [52]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [53]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [54]:
# le = LabelEncoder()
# cols = ['Geography', 'Gender']

# df[cols] = df[cols].apply(le.fit_transform)

In [55]:
X = df.iloc[:, 3: -1].to_numpy()
y = df.iloc[:, -1].to_numpy()
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

### Encode catagorical data

In [56]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [57]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

### Split into training and test set

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

### Feature Scaling

Vary important when creating Neural Networks

In [59]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

 Initialize the ANN with the [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) method from TensorFlow Keras API

In [60]:
from tensorflow.keras import Sequential
model = Sequential()

# also same as:
# model = tf.keras.models.Sequential() - Since the Sequential() method inherits from Model

Add the input layer and the first hidden layer

`(units, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)`

In [61]:
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the second hidden layer

In [62]:
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

Add the output layer

In [63]:
model.add(tf.keras.layers.Dense(units=1, activation='relu'))

In [64]:
# all that adding layers same as:

# from tensorflow.keras.layers import Dense

# model = Sequential([
#     Dense(units=6, activation='relu'),
#     Dense(units=6, activation='relu'),
#     Dense(units=1, activation='relu')    
# ])

# Training the ANN

First we compile the model

model.compile(
    `optimizer='rmsprop',
    loss=None,
    metrics=None,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    jit_compile=None,
    **kwargs`
)

We commoly use the `adam`(Adaptive Moement estimation) optimizer as it requires less memory and is efficient.

The gradient descent method is upgraded for the optimization tasks

In [65]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the model

model.fit(
  `
  train_data,
  steps_per_epoch = train_samples//batch_size,
  epochs = epochs,
  validation_data = test_data,
  verbose = 1,
  validation_steps = test_samples//batch_size`
  
)

* `epochs` is the number of times the algorithm will work through the entire training data set.

In [66]:
%%time
model.fit(X_train, y_train, batch_size=32, epochs= 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.9833 - accuracy: 0.7427
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5566 - accuracy: 0.7896
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5099 - accuracy: 0.8014
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4802 - accuracy: 0.8064
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5119 - accuracy: 0.7924
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4758 - accuracy: 0.8025
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4656 - accuracy: 0.8059
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4568 - accuracy: 0.8084
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4526 - accuracy: 0.8115
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4492 - accura

## Making Predictions

Now let's try to predict with our model if the customer with the folloiwng information will leave the bank or not

* Geography - France
* Credit Score - 600
* Gender - Male
* Age - 40
* Tenure - 3 years
* Balance - $ 60000
* Number of Products - 2
* Has Credit Card? yes
* Active Member? Yes
* Estimated Salary -  50000 dollars




In [67]:
# Returns a probability
prediction = model.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5
print('Prediction: ', prediction[0][0])

1/1 [==============================] - 0s 67ms/step
Prediction:  False
